In [35]:
import pandas as pd
import re

In [36]:
df = pd.read_excel("PAN Number Validation Dataset.xlsx")
df.head(10)
total_records = len(df)

**Data Preprocessing**

In [37]:
df.shape

(10000, 1)

In [38]:
df.describe()

,Pan_Numbers
count,9035
unique,9027
top,JVPYR52307F
freq,4


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Pan_Numbers  9035 non-null   object
dtypes: object(1)
memory usage: 78.3+ KB


**Data Cleaning**

In [40]:
# removing extra spaces from the data and converting all the data into uppercase
df['Pan_Numbers'] = df['Pan_Numbers'].astype('string').str.strip().str.upper()
df.head(10)

,Pan_Numbers
0,VGLOD3180G
1,PHOXD7232L
2,MGEPH6532A
3,JJCHK4574O
4,XTQIJ2330L
5,HTJYM3835H
6,YQTAP6661X
7,HVOFE5635Y
8,HYUIJ7902R
9,IDSMT3429E


In [41]:
# removing null and empty values from the data
df = df.replace({'Pan_Numbers':''}, pd.NA).dropna(subset='Pan_Numbers')
print('Total no of rows:', len(df))

Total no of rows: 9033


In [42]:
# removing duplicate values

In [43]:
print('Unique values in dataframe:', df['Pan_Numbers'].nunique())

Unique values in dataframe: 9025


In [44]:
df = df.drop_duplicates(subset='Pan_Numbers', keep='first')
print('Total no of rows:', len(df))

Total no of rows: 9025


**Data Validation**

In [45]:
'''
def has_adjacent_repitition(pan):
    for i in range(len(pan)-1):
        if pan[i] == pan[i+1]:
            return True
    return False

def is_sequencial(pan):
    for i in range(len(pan)-1):
        if ord(pan[i+1]) - ord(pan[i]) != 1:
            return False
    return True
'''

# other way to write the same code using list comp
def has_adjacent_repitition(pan):
    return any(pan[i] == pan[i+1] for i in range(len(pan)-1))

def is_sequencial(pan):
    return all(ord(pan[i+1]) - ord(pan[i]) == 1 for i in range(len(pan)-1))

In [46]:
def is_valid_pan(pan):
    if len(pan) != 10:
        return False

    if not re.match(r'^[A-Z]{5}[0-9]{4}[A-Z]{1}', pan):
        return False

    if has_adjacent_repitition(pan):
        return False

    if is_sequencial(pan):
        return False

    return True

** Categorizing the data**

In [48]:
df['Status'] = df['Pan_Numbers'].apply(lambda x: "Valid" if is_valid_pan(x) else "Invalid")
df.head(10)

,Pan_Numbers,Status
0,VGLOD3180G,Valid
1,PHOXD7232L,Valid
2,MGEPH6532A,Valid
3,JJCHK4574O,Invalid
4,XTQIJ2330L,Invalid
5,HTJYM3835H,Valid
6,YQTAP6661X,Invalid
7,HVOFE5635Y,Valid
8,HYUIJ7902R,Valid
9,IDSMT3429E,Valid


**Data Summary**

In [49]:
valid_cnt = (df['Status']=='Valid').sum()
invalid_cnt = (df['Status'] == 'Invalid').sum()
missing_cnt = total_records - (valid_cnt + invalid_cnt)

In [50]:
print("Total records = ",total_records)
print("No of Valid PAN = ",valid_cnt)
print("No of Invalid PAN = ",invalid_cnt)
print("No of missing PAN = ",missing_cnt)

Total records =  10000
No of Valid PAN =  3193
No of Invalid PAN =  5832
No of missing PAN =  975


**Importing data to Excel**

In [51]:
df_summary = pd.DataFrame({"TOTAL PROCESSED RECORDS":[total_records],
                           "TOTAL VALID COUNT": [valid_cnt],
                           "TOTAL INVALID COUNT": [invalid_cnt],
                           "TOTAL MISSING PAN": [missing_cnt]
                          })
df_summary

,TOTAL PROCESSED RECORDS,TOTAL VALID COUNT,TOTAL INVALID COUNT,TOTAL MISSING PAN
0,10000,3193,5832,975


In [52]:
with pd.ExcelWriter("PAN VALIDATION RESULT.xlsx") as writer:
    df.to_excel(writer, sheet_name="PAN Validations", index=False)
    df_summary.to_excel(writer, sheet_name="SUMMARY", index=False)